## Import Required Libraries

In [1]:
#Neural Network modules
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical


import numpy                    #Module for working with arrays
import os                       #Standard module for interacting with OS
import random                   #For shuffling images
import cv2                      #Standard module in solving computer visions
from scipy import ndimage       #Library for multidimensional image processing

## Function to calculate the COM shift

In [2]:
def getBestShift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)
    rows,cols = img.shape
    shiftx = numpy.round(cols/2.0-cx).astype(int)
    shifty = numpy.round(rows/2.0-cy).astype(int)
    return shiftx,shifty

## Function to shift image

In [3]:
def shift(img,sx,sy):
    rows,cols = img.shape
    M = numpy.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

## Function to centralize image

In [4]:
def shiftCenterOfMass(img):
    img = cv2.bitwise_not(img)
    # Centralize the image according to center of mass
    shiftx,shifty = getBestShift(img)
    shifted = shift(img,shiftx,shifty)
    img = shifted
    img = cv2.bitwise_not(img)
    return img

## Function to create training data

In [5]:
def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (img_rows, img_cols))
            new_array = shiftCenterOfMass(new_array)
            training_data.append([new_array, class_num])

In [6]:
batch_size = 128
num_classes = 9
epochs = 45

# input image dimensions
img_rows, img_cols = 28, 28

DATADIR = "DigitImages"
CATEGORIES = ["1","2","3","4","5","6","7","8","9"]

training_data = []

createTrainingData()

# Mix data up
random.shuffle(training_data)

# Split 80-20
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(len(training_data)*8//10):
    x_train.append(training_data[i][0])
    y_train.append(training_data[i][1])
for i in range(len(training_data)*8//10,len(training_data)):
    x_test.append(training_data[i][0])
    y_test.append(training_data[i][1])

# Reshape
x_train = numpy.array(x_train)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = numpy.array(x_test)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize the data
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer="Adam",metrics=['accuracy'])

model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save_weights('digitRecognition.h5')

C:\Users\Aman Yadav\AppData\Local\Temp\ipykernel_17540\1076359088.py:2: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  cy,cx = ndimage.measurements.center_of_mass(img)


Epoch 1/45
4/4 [==============================] - 3s 313ms/step - loss: 2.1928 - accuracy: 0.1256 - val_loss: 2.1739 - val_accuracy: 0.1224
Epoch 2/45
4/4 [==============================] - 1s 176ms/step - loss: 2.1293 - accuracy: 0.1923 - val_loss: 2.1845 - val_accuracy: 0.0714
Epoch 3/45
4/4 [==============================] - 1s 254ms/step - loss: 2.0595 - accuracy: 0.2179 - val_loss: 2.1452 - val_accuracy: 0.1939
Epoch 4/45
4/4 [==============================] - 1s 192ms/step - loss: 1.9747 - accuracy: 0.3231 - val_loss: 2.0726 - val_accuracy: 0.2755
Epoch 5/45
4/4 [==============================] - 1s 167ms/step - loss: 1.9169 - accuracy: 0.3641 - val_loss: 2.0220 - val_accuracy: 0.2857
Epoch 6/45
4/4 [==============================] - 1s 168ms/step - loss: 1.7963 - accuracy: 0.4462 - val_loss: 1.9627 - val_accuracy: 0.3980
Epoch 7/45
4/4 [==============================] - 1s 186ms/step - loss: 1.7186 - accuracy: 0.4718 - val_loss: 1.8605 - val_accuracy: 0.4694
Epoch 8/45
4/4 [====

In [7]:
import h5py 
#Open the H5 file in read mode
with h5py.File('digitRecognition.h5', 'r') as file:
    print("Keys: %s" % file.keys())

Keys: <KeysViewHDF5 ['conv2d', 'conv2d_1', 'dense', 'dense_1', 'dropout', 'dropout_1', 'flatten', 'max_pooling2d', 'top_level_model_weights']>
